# **Practice**:
### **Theme**: 
##### **Date**: 08.11.24

In [ ]:
def findMaxFlow(self, n, m, edges):
    matr = [[0] * n for _ in range(n)]
    for i, j, c in edges:
        matr[i - 1][j - 1] += c
        matr[j - 1][i - 1] += c

    p = [-1] * n

    def bfs():
        v = [0] * n
        q = [0]
        v[0] = 1

        while q:
            u = q.pop(0)

            for i, c in enumerate(matr[u]):
                if not v[i] and c > 0:
                    q.append(i)
                    p[i] = u
                    v[i] = 1
                    if i == n - 1:
                        return True
        return False

    max_flow = 0
    while bfs():
        path_flow = float("inf")
        v = n - 1

        while v != 0:
            u = p[v]
            path_flow = min(path_flow, matr[u][v])
            v = u

        max_flow += path_flow
        v = n - 1
        while v != 0:
            u = p[v]
            matr[u][v] -= path_flow
            matr[v][u] += path_flow
            v = u

    return max_flow


In [ ]:
from typing import List

class Solution:
    def isCyclic(self, V: int, adj: List[List[int]]) -> bool:
        visited = [False] * V
        recStack = [False] * V

        def dfs(u):
            visited[u] = True
            recStack[u] = True

            for neighbor in adj[u]:
                if not visited[neighbor]:
                    if dfs(neighbor):
                        return True
                elif recStack[neighbor]:
                    return True

            recStack[u] = False
            return False


        for u in range(V):
            if not visited[u]:
                if dfs(u):
                    return True

        return False


In [2]:
from typing import List

class Solution:
    def isCyclic(self, V: int, adj: List[List[int]]) -> bool:
        in_deg = [0] * V
        for i in range(V):
            for j in adj[i]:
                in_deg[j] += 1

        q = []
        t_sort = []
        for i, v in enumerate(in_deg):
            if not v:
                q.append(i)

        while q:
            u = q.pop(0)
            t_sort.append(u)

            for v in adj[u]:
                in_deg[v]  -= 1
                if not in_deg[v]:
                    q.append(v)

        if len(t_sort) == V:
            return 0
        else:
            return 1


In [ ]:
def isNegativeWeightCycle(self, n, edges):
    dist = [float("inf")] * n
    dist[0] = 0

    for _ in range(n - 1):
        for u, v, w in edges:
            if dist[u] != float("inf") and dist[u] + w < dist[v]:
                dist[v] = dist[u] + w

    for u, v, w in edges:
        if dist[u] != float("inf") and dist[u] + w < dist[v]:
            return 1

    return 0
